In [ ]:
!git clone https://github.com/wkdgnsgo/kmong_dit.git
!pip install -q omegaconf
!pip install -q pytorch-lightning
!pip install -q taming-transformers-rom1504
!pip install -q kornia
!pip install -q openai-clip

Cloning into 'kmong_dit'...
remote: Enumerating objects: 2519, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2519 (delta 14), reused 13 (delta 9), pack-reused 2489 (from 4)
Receiving objects: 100% (2519/2519), 85.75 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (440/440), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
from omegaconf import OmegaConf
import os, sys, datetime, glob, importlib, csv
import random
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# import sys, types
# mpi4py = types.ModuleType("mpi4py")
# class _DummyComm:
#     def Get_size(self): return 1
#     def Get_rank(self): return 0
#     def bcast(self, x, root=0): return x
# class _DummyMPI: COMM_WORLD = _DummyComm()
# mpi4py.MPI = _DummyMPI()
# sys.modules["mpi4py"] = mpi4py

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#unet_config:
# params:
#         image_size: 128
#         in_channels: 6
#         out_channels: 3
#         model_channels: 128
#         attention_resolutions:
#         - 32
#         - 16
#         - 8
#         num_res_blocks: 2
#         channel_mult:
#         - 1
#         - 4
#         - 8
# data:
#  train:
#       target: ldm.data.kvasir.KvasirSegTrain
#       params:
#         size: 512
#     validation:
#       target: ldm.data.kvasir.KvasirSegEval
#       params:
#         size: 512

configs = [OmegaConf.load(cfg) for cfg in ["/content/kmong_dit/latent-diffusion/configs/latent-diffusion/kvasir-flow-matching.yaml"]]

In [ ]:
# data_train.txt, data_eval.txt 가 존재하지 않는다면, 이 cell을 실행해서 생성해야됌
data_root = "/content/drive/MyDrive/01_data/"
data_set_name = ""

data_path = os.path.join(data_root, data_set_name, "images")

def create_data_csv(data_path, ratio=0.8):
    data_list = os.listdir(data_path)
    random.shuffle(data_list)
    train_list = data_list[:int(len(data_list)*ratio)]
    test_list = data_list[int(len(data_list)*ratio):]

    with open(os.path.join("/content/drive/MyDrive/01_data/", "data_train.txt"), "w") as f: # path
        for item in train_list:
            f.write(item+"\n")

    with open(os.path.join("/content/drive/MyDrive/01_data/", "data_eval.txt"), "w") as f: # path
        for item in test_list:
            f.write(item+"\n")

create_data_csv(data_path)

In [ ]:
# latent-diffusion/ldm/data/kvasir에서 KvasirSegTrain와 KvasirSegEval 수정

# class KvasirSegTrain(SegmentationBase):
#     def __init__(self, size=None, random_crop=False, interpolation="bicubic"):
#         super().__init__(data_csv='/content/data_train.txt', # data_train.txt가 있는 위치 수정
#                          data_root='/content/drive/MyDrive/velum/images', # 데이터 있는 위치로 수정
#                          segmentation_root='/content/drive/MyDrive/velum/masks', # 데이터 있는 위치로 수정
#                          size=size, random_crop=random_crop, interpolation=interpolation,
#                          n_labels=2)

# class KvasirSegEval(SegmentationBase):
#     def __init__(self, size=None, random_crop=False, interpolation="bicubic"):
#         super().__init__(data_csv='/content/data_eval.txt', # data_eval.txt가 있는 위치로 수정
#                          data_root='/content/drive/MyDrive/velum/images', # 데이터 있는 위치로 수정
#                          segmentation_root='/content/drive/MyDrive/velum/masks', # 데이터 있는 위치로 수정
#                          size=size, random_crop=random_crop, interpolation=interpolation,
#                          n_labels=2)

In [ ]:
import numpy as np
print(np.__version__)

1.26.4


In [ ]:
%cd kmong_dit/latent-diffusion/

/content/kmong_dit/latent-diffusion


In [ ]:
# from conditional-polyp-diffusion/latent-diffusion/ldm/models/diffusion/ddpm.py 교체
# main.py 파일 17번째 줄 아래와 같이 수정
# from pytorch_lightning.utilities import rank_zero_only
from ldm.util import instantiate_from_config
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

model = instantiate_from_config(configs[0].model)
model = model.to("cuda")
# model.logvar = model.logvar.to("cuda")
model.learning_rate = 1.0e-04

making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 128, 128) = 49152 dimensions.
making attention of type 'vanilla' with 512 in_channels
Restored from /content/drive/MyDrive/VQVAE-epoch=105.ckpt with 0 missing and 55 unexpected keys


In [ ]:
# data loader setup
data = instantiate_from_config(configs[0].data)
data.prepare_data()
data.setup()
print("#### Data #####")
for k in data.datasets:
    print(f"{k}, {data.datasets[k].__class__.__name__}, {len(data.datasets[k])}")

#### Data #####
train, KvasirSegTrain, 28
validation, KvasirSegEval, 7


In [ ]:
import pytorch_lightning as pl
print(pl.__version__)

2.6.0


In [ ]:
saving_ckpt_path = "/content/"

checkpoint_callback = ModelCheckpoint(
    dirpath=saving_ckpt_path,  # Directory to save checkpoints
    filename='latent_diffusion',  # Filename format
    monitor='val/loss',  # Metric to monitor
    mode='min',          # Mode for monitoring ('min' or 'max')
    save_top_k=3         # Number of top models to save
)

# trainer = Trainer(
#     accelerator="gpu",
#     devices=1,
#     strategy=None,
#     max_epochs=1000,
#     callbacks=[checkpoint_callback]
# )

trainer = Trainer(
    accelerator = "gpu" if torch.cuda.is_available() else "cpu",
    devices     = 1,              # 또는 devices=[0]
    # strategy    = None,         # None 대신 "auto"
    max_epochs  = 1000,
    callbacks   = [checkpoint_callback],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, data)

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1551: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return _C._get_float32_matmul_precision()
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_p

┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name              ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ first_stage_model │ VQModelInterface │ 55.3 M │ eval  │     0 │
│ 1 │ model             │ LightningDiT     │  457 M │ train │     0 │
└───┴───────────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 456 M                                                                                            
Non-trainable params: 55.6 M                                                                                       
Total params: 512 M                                                                                                
Total estimated model params size (MB): 2.0 K                                                                      
Modules in train mode: 424                                                                                         
Modules in eval mode: 174                                                                                          
Total FLOPs: 0

Output()